<p style="border: 1px solid #e7692c; border-left: 15px solid #e7692c; padding: 10px; text-align:justify;">
    <strong style="color: #e7692c">Tip.</strong> <a style="color: #000000;" href="https://nbviewer.jupyter.org/github/PacktPublishing/Hands-On-Computer-Vision-with-TensorFlow-2/blob/master/Chapter09/ch9_nb3_train_model.ipynb" title="View with Jupyter Online">Click here to view this notebook on <code>nbviewer.jupyter.org</code></a>. 
    <br/>These notebooks are better read there, as Github default viewer ignores some of the formatting and interactive content.
    </p>

<table style="font-size: 1em; padding: 0; margin: 0;">
    <tr style="vertical-align: top; padding: 0; margin: 0;background-color: #ffffff">
        <td style="vertical-align: top; padding: 0; margin: 0; padding-right: 15px;">
    <p style="background: #363636; color:#ffffff; text-align:justify; padding: 10px 25px;">
        <strong style="font-size: 1.0em;"><span style="font-size: 1.2em;"><span style="color: #e7692c;">Hands-on</span> Computer Vision with TensorFlow 2</span><br/>by <em>Eliot Andres</em> & <em>Benjamin Planche</em> (Packt Pub.)</strong><br/><br/>
        <strong>> Chapter 9: Performance and running on mobile</strong><br/>
    </p>

<h1 style="width: 100%; text-align: left; padding: 0px 25px;"><small style="color: #e7692c;">Notebook 3:</small><br/>Training a model and converting it for mobile devices</h1>
<br/>
<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    In this chapter, we covered how to convert and run a model on mobile.
<br/><br/>
    This notebooks trains a model to recognize face expressions and converts it to CoreML, TFLite and TensorFlow.js
</p>
<br/>

<p style="border-left: 15px solid #363636; text-align:justify; padding: 0 10px;">
    <strong> Requirements </strong>
<br/><br/>
    To run this notebook, you need to download the <a href="https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data">FER dataset</a> and extract it. When done, change the `BASE_PATH` variable to point to the dataset folder.
</p>
<br/>
<p style="border-left: 15px solid #e7692c; padding: 0 10px; text-align:justify;">
    <strong style="color: #e7692c;">Tip.</strong> The notebooks shared on this git repository illustrate some notions from the book "<em><strong>Hands-on Computer Vision with TensorFlow 2</strong></em>" written by Eliot Andres and Benjamin Planche, published by Packt. If you enjoyed the insights shared here, <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Learn more about the book!"><strong>please consider acquiring the book!</strong></a>
<br/><br/>
The book provides further guidance for those eager to learn about computer vision and to harness the power of TensorFlow 2 and Keras to build efficient recognition systems for object detection, segmentation, video processing, smartphone applications, and more.</p>
        </td>
        <td style="vertical-align: top; padding: 0; margin: 0; width: 280px;">
    <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Learn more about the book!" target="_blank">
        <img src="../banner_images/book_cover.png" width=280>
    </a>
    <p style="background: #e7692c; color:#ffffff; padding: 10px; text-align:justify;"><strong>Leverage deep learning to create powerful image processing apps with TensorFlow 2 and Keras. <br/></strong>Get the book for more insights!</p>
    <ul style="height: 32px; white-space: nowrap; text-align: center; margin: 0px; padding: 0px; padding-top: 10px;">
    <li style="display: block;height: 100%;float: left;vertical-align: middle;margin: 0 25px 10px;padding: 0px;">
        <a href="https://www.amazon.com/Hands-Computer-Vision-TensorFlow-processing/dp/1788830644" title="Get the book on Amazon (paperback or Kindle version)!" target="_blank">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_amazon.png" width="75px">
        </a>
    </li>
    <li style="display: inline-block;height: 100%;vertical-align: middle;float: right;margin: -5px 25px 10px;padding: 0px;">
        <a href="https://www.packtpub.com/application-development/hands-computer-vision-tensorflow-2" title="Get your Packt book (paperback, PDF, ePUB, or MOBI version)!" target="_blank">
        <img style="vertical-align: middle; max-width: 72px; max-height: 32px;" src="../banner_images/logo_packt.png" width="75px">
        </a>
    </li>
    </ul>
        </td>
        </tr>
        </table>

In [1]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf

tf.__version__

'2.0.0-dev20190428'

## Set constants and parameters


In [2]:
# Download data here: https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data
DATASET_PATH = '../data/fer2013/fer2013.csv'

IMAGE_SIZE = (48, 48)
INPUT_SHAPE = IMAGE_SIZE + (1,)
EMOTIONS = ["angry", "disgust", "scared",
            "happy", "sad", "surprised", "neutral"]
CALLBACK_PATIENCE = 50
BATCH_SIZE = 32
NUM_EPOCHS = 50
VALIDATION_SPLIT = .2
NUM_CLASSES = len(EMOTIONS)
L2_REGULARIZATION = 0.01

## Build the model

In [3]:
input_tensor = Input(shape=INPUT_SHAPE)
model = MobileNet(input_tensor=input_tensor, alpha=1.0,
                    include_top=False, weights=None)

output = tf.keras.layers.Reshape((1024,))(model.output)
output = tf.keras.layers.Dense(7, activation='softmax')(output)
model = tf.keras.Model(model.input, output)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 49, 49, 1)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 24, 24, 32)        288       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 24, 24, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 24, 24, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 24, 24, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 24, 24, 32)        128   

## Load data

In [4]:
def load_fer2013():
    data = pd.read_csv(DATASET_PATH)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), IMAGE_SIZE)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    x = x - 0.5
    x = x * 2.0
    return x

data_generator = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    horizontal_flip=True)

# loading dataset
faces, emotions = load_fer2013()
faces = preprocess_input(faces)


## Train model

In [9]:
regularization = tf.keras.regularizers.l2(L2_REGULARIZATION)

early_stop = EarlyStopping('val_loss', patience=CALLBACK_PATIENCE)
reduce_lr = ReduceLROnPlateau(
    'val_loss', factor=0.1, patience=int(CALLBACK_PATIENCE/4), verbose=1)
tensorboard = TensorBoard('./logs')
callbacks = [early_stop, reduce_lr, tensorboard]

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
xtrain, xtest, ytrain, ytest = train_test_split(faces, emotions, test_size=0.2)

model.fit_generator(data_generator.flow(xtrain, ytrain, BATCH_SIZE),
                    steps_per_epoch=len(xtrain) / BATCH_SIZE,
                    epochs=NUM_EPOCHS, verbose=1, callbacks=callbacks,
                    validation_data=(xtest, ytest))

Epoch 1/50
898/897 [==============================] - 37s 41ms/step - loss: 1.8936 - accuracy: 0.2319 - val_loss: 1.9043 - val_accuracy: 0.2530
Epoch 2/50
898/897 [==============================] - 21s 24ms/step - loss: 1.8200 - accuracy: 0.2649 - val_loss: 1.7767 - val_accuracy: 0.2768
Epoch 3/50
898/897 [==============================] - 21s 24ms/step - loss: 1.7820 - accuracy: 0.2988 - val_loss: 1.8422 - val_accuracy: 0.3034
Epoch 4/50
898/897 [==============================] - 21s 24ms/step - loss: 1.7192 - accuracy: 0.3265 - val_loss: 1.9029 - val_accuracy: 0.2336
Epoch 5/50
898/897 [==============================] - 21s 24ms/step - loss: 1.6862 - accuracy: 0.3394 - val_loss: 1.6631 - val_accuracy: 0.3416
Epoch 6/50
898/897 [==============================] - 21s 24ms/step - loss: 1.6721 - accuracy: 0.3502 - val_loss: 1.6184 - val_accuracy: 0.3767
Epoch 7/50
898/897 [==============================] - 21s 24ms/step - loss: 1.5909 - accuracy: 0.3841 - val_loss: 1.6010 - val_accuracy:

## Convert to CoreML

In [10]:
import tfcoreml as tf_converter
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.tools import freeze_graph
input_saved_model_dir = "./saved_model"

tf.keras.experimental.export_saved_model(
    model, saved_model_path=input_saved_model_dir, serving_only=False)


output_node_name = 'dense/Softmax'
input_binary = False
input_saver_def_path = False
restore_op_name = None
filename_tensor_name = None
clear_devices = True
input_meta_graph = False
checkpoint_path = None
input_graph_filename = None
saved_model_tags = tag_constants.SERVING


freeze_graph.freeze_graph(input_graph_filename, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_name,
                          restore_op_name, filename_tensor_name,
                          'frozen_model.pb', clear_devices, "", "", "",
                          input_meta_graph, input_saved_model_dir,
                          saved_model_tags)


tf_converter.convert('frozen_model.pb',
                     'mobilenet.mlmodel',
                     class_labels=EMOTIONS,
                     image_input_names=['input_1:0'],
                     output_feature_names=[output_node_name + ':0'],
                     red_bias=-1,
                     green_bias=-1,
                     blue_bias=-1,
                     image_scale=1/127.5,
                     is_bgr=False)

W0526 16:34:32.567987 140384533387008 tf_logging.py:161] Export includes no default signature!
W0526 16:34:35.210698 140384533387008 tf_logging.py:161] Export includes no default signature!



Loading the TF graph...
Graph Loaded.
Now finding ops in the TF graph that can be dropped for inference
Done.
Now starting translation to CoreML graph.
Automatic shape interpretation succeeded for input blob input_1:0
1/377: Analysing op name: dense/bias ( type:  Const )
2/377: Analysing op name: dense/BiasAdd/ReadVariableOp ( type:  Identity )
3/377: Analysing op name: dense/kernel ( type:  Const )
4/377: Analysing op name: dense/MatMul/ReadVariableOp ( type:  Identity )
5/377: Analysing op name: reshape/Reshape/shape/1 ( type:  Const )
6/377: Analysing op name: reshape/strided_slice/stack_2 ( type:  Const )
7/377: Analysing op name: reshape/strided_slice/stack_1 ( type:  Const )
8/377: Analysing op name: reshape/strided_slice/stack ( type:  Const )
9/377: Analysing op name: conv_pw_13_bn/moving_variance ( type:  Const )
10/377: Analysing op name: conv_pw_13_bn/FusedBatchNorm/ReadVariableOp_1 ( type:  Identity )
11/377: Analysing op name: conv_pw_13_bn/moving_mean ( type:  Const )
12

326/377: Analysing op name: conv_pw_6_bn/FusedBatchNorm ( type:  FusedBatchNorm )
327/377: Analysing op name: conv_pw_6_relu/Relu6 ( type:  Relu6 )
328/377: Analysing op name: conv_dw_7/depthwise ( type:  DepthwiseConv2dNative )
329/377: Analysing op name: conv_dw_7_bn/FusedBatchNorm ( type:  FusedBatchNorm )
330/377: Analysing op name: conv_dw_7_relu/Relu6 ( type:  Relu6 )
331/377: Analysing op name: conv_pw_7/Conv2D ( type:  Conv2D )
332/377: Analysing op name: conv_pw_7_bn/FusedBatchNorm ( type:  FusedBatchNorm )
333/377: Analysing op name: conv_pw_7_relu/Relu6 ( type:  Relu6 )
334/377: Analysing op name: conv_dw_8/depthwise ( type:  DepthwiseConv2dNative )
335/377: Analysing op name: conv_dw_8_bn/FusedBatchNorm ( type:  FusedBatchNorm )
336/377: Analysing op name: conv_dw_8_relu/Relu6 ( type:  Relu6 )
337/377: Analysing op name: conv_pw_8/Conv2D ( type:  Conv2D )
338/377: Analysing op name: conv_pw_8_bn/FusedBatchNorm ( type:  FusedBatchNorm )
339/377: Analysing op name: conv_pw_8_

input {
  name: "input_1__0"
  type {
    imageType {
      width: 48
      height: 48
      colorSpace: GRAYSCALE
    }
  }
}
output {
  name: "dense__Softmax__0"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "dense__Softmax__0"

## Convert to TFLite

In [11]:

converter = tf.lite.TFLiteConverter.from_keras_model(model)
## Or from a SavedModel
# converter = tf.lite.TFLiteConverter('./saved_model')

tflite_model = converter.convert()
open("result.tflite", "wb").write(tflite_model)



12784696

## Convert to TFJS

In [14]:
# Convert in the current environment
import sys
!tensorflowjs_converter --input_format=tf_saved_model saved_model my-tfjs --output_format tfjs_graph_model